# Utiliza o CDB com ML para predizer a localização (lat,lon) das EM:

In [10]:
import pandas as pd
import numpy as np
import math
import os
from tqdm import tqdm
%run Imports.ipynb
%run UtilsLightGBMFingerPrinting.ipynb

In [11]:
FILES = [
    # Arquivos de Teste Outdoor Só possui RSSIs, delays, idx
    #(7,'./Test/Outdoor_Test.csv'), # Usar CDB_20_LightGBM_Complete
    #(8,'./Test/Indoor_Test-fold.csv'), # Usar CDB_20_LightGBM_Complete # 
    #(9,'./Test/Indoor-Outdoor_Test.csv'), # Usar CDB_20_LightGBM_Complete
    
    (7,'./k-fold-LightGBM/Test-LightGBM-Outdoor-Fold1.csv'), # Usar CDB_20_LightGBM_Complete_k-fold 
    (8,'./k-fold-LightGBM/Indoor_Test.csv'), # Usar CDB_20_LightGBM_Complete_k-fold
    (9,'./k-fold-LightGBM/Test-LightGBM-Indoor-Outdoor-Fold1.csv') # Usar CDB_20_LightGBM_Complete_k-fold
    
    #(10,'./Test/Outdoor_Test.csv'), # Usar CDB_20_SVR_Complete
    #(11,'./Test/Indoor_Test-fold.csv'), # Usar CDB_20_SVR_Complete 
    #(12,'./Test/Indoor-Outdoor_Test.csv') # Usar CDB_20_SVR_Complete
    
    #(10,'./k-fold-SVR/Test-SVR-Outdoor-Fold4.csv'), # Usar CDB_20_SVR_Complete_k-fold 
    #(11,'./k-fold-SVR/Indoor_Test.csv'), # Usar CDB_20_SVR_Complete_k-fold
    #(12,'./k-fold-SVR/Test-SVR-Indoor-Outdoor-Fold4.csv') # Usar CDB_20_SVR_Complete_k-fold
]

#TEST_DATABASE = "./CDB/CDB_20_LightGBM_Complete.csv"
#TEST_DATABASE = "./CDB/CDB_20_SVR_Complete.csv"

#TEST_DATABASE = "./k-fold-SVR/CDB-Fold4-SVR.csv"
TEST_DATABASE = "./k-fold-LightGBM/CDB-Fold1-LightGBM.csv"

# Sem k-fold:

In [12]:
generate_theorical_results(FILES, TEST_DATABASE)

400it [00:12, 32.36it/s]
400it [00:12, 32.03it/s]
400it [00:12, 32.55it/s]


In [25]:
x = np.arange(1,11)
x

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [81]:
point = ["rssi_1_1","rssi_1_2","rssi_1_3","rssi_2_1","rssi_2_2","rssi_2_3","rssi_3_1","rssi_3_2","rssi_3_3","delay_1","delay_2","delay_3","delay_12","delay_13","delay_23","idx"]
point[:-7]#-1

['rssi_1_1',
 'rssi_1_2',
 'rssi_1_3',
 'rssi_2_1',
 'rssi_2_2',
 'rssi_2_3',
 'rssi_3_1',
 'rssi_3_2',
 'rssi_3_3']

In [82]:
cdb = ["rssi_1_1","rssi_1_2","rssi_1_3","rssi_2_1","rssi_2_2","rssi_2_3","rssi_3_1","rssi_3_2","rssi_3_3","delay_1","delay_2","delay_3","delay_12","delay_13","delay_23","lat","lon"]
cdb[:-8]#-2

['rssi_1_1',
 'rssi_1_2',
 'rssi_1_3',
 'rssi_2_1',
 'rssi_2_2',
 'rssi_2_3',
 'rssi_3_1',
 'rssi_3_2',
 'rssi_3_3']